In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier as dt


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
train = pd.read_csv('datathon2020_train.csv')
train

,ID,F2,F3,F4,F8,F11,F25,F39,F52,F53,...,F127,F128,F129,F130,F131,F132,F133,F136,F139,COVID_TCPM
0,10681,1,1,1,Tropical and Subtropical Moist Broadleaf Forests,"Mild temperate with dry winter, and Warm summer",9.028250e+04,0.367070,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,available,0.102565,low
1,3300,1,1,1,Temperate Broadleaf and Mixed Forests,"Mild temperate, fully humid, and Hot summer",2.885228e+05,0.420663,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,available,0.186848,high
2,12038,1,1,1,Temperate Broadleaf and Mixed Forests,"Mild temperate, fully humid, and Hot summer",9.469753e+04,0.343825,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,available,0.131743,low
3,10236,0,1,1,Tropical and Subtropical Moist Broadleaf Forests,Tropical monsoon,1.753151e+05,0.558702,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,preliminary,0.703491,low
4,4111,1,1,1,Temperate Broadleaf and Mixed Forests,"Snow, fully humid, and Warm summer",8.909464e+04,0.508010,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,available,0.531441,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,5740,0,1,1,Deserts and Xeric Shrublands,"Desert (arid), and Hot arid",2.051100e+05,0.094838,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,available,0.000000,high
496,554,1,1,1,Temperate Broadleaf and Mixed Forests,"Snow, fully humid, and Hot summer",1.733145e+06,0.515650,2.631636e+06,2.385705e+06,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,available,0.731324,high
497,12566,1,1,1,Tropical and Subtropical Moist Broadleaf Forests,"Mild temperate, fully humid, and Hot summer",3.038501e+06,0.315763,1.613929e+05,3.182036e+05,...,0.946529,1.200975,1.281726,7113.034668,8993.411865,10894.03711,12675.01086,available,0.034969,low
498,13005,0,1,1,Tropical and Subtropical Moist Broadleaf Forests,Tropical rain forest,7.359616e+05,0.439870,3.912408e+01,6.799629e+02,...,16.289423,19.483387,20.412186,88959.949020,281104.611600,456585.33630,735961.58750,preliminary,0.342686,low


In [3]:
test = pd.read_csv('datathon2020_test.csv')
test

,ID,F2,F3,F4,F8,F11,F25,F39,F52,F53,...,F127,F128,F129,F130,F131,F132,F133,F136,F139,COVID_TCPM
0,12272,1,1,1,Temperate Broadleaf and Mixed Forests,"Mild temperate, fully humid, and Hot summer",3.010114e+05,0.270918,NaN,526075.1429,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.008360,low
1,6526,1,1,1,Deserts and Xeric Shrublands,"Desert (arid), and Hot arid",8.668036e+04,0.303313,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.000000,low
2,11320,2,1,1,Temperate Broadleaf and Mixed Forests,"Mild temperate with dry winter, and Hot summer",2.972672e+05,0.359829,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.165453,low
3,4566,0,0,1,Temperate Broadleaf and Mixed Forests,"Mild temperate with dry summer, and Hot summer",9.294967e+04,0.274426,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.029401,low
4,4866,1,1,1,Tropical and Subtropical Moist Broadleaf Forests,"Mild temperate, fully humid, and Warm summer",1.546434e+05,0.475473,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.334338,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481,1575,1,1,1,"Mediterranean Forests, Woodlands, and Scrub","Mild temperate with dry summer, and Hot summer",3.985660e+06,0.262157,168320.0651,367945.1962,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.040463,low
2482,13009,0,0,1,Tropical and Subtropical Moist Broadleaf Forests,Tropical rain forest,7.704883e+04,0.388311,NaN,NaN,...,2.593406,2.593406,2.688027,13306.76338,37205.58549,55806.48722,77048.82957,preliminary,0.221624,low
2483,11032,0,1,1,Tropical and Subtropical Moist Broadleaf Forests,Tropical rain forest,5.288064e+04,0.556476,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.794566,low
2484,7213,0,1,1,Tropical and Subtropical Dry Broadleaf Forests,Tropical savannah with dry winter,1.618023e+05,0.405874,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.107804,low


In [4]:
train.isna().sum()

ID              0
F2              0
F3              0
F4              0
F8              0
F11             0
F25             0
F39            14
F52           364
F53           350
F54           349
F55           328
F69            23
F72           453
F73           437
F74           418
F75           389
F76             3
F77             3
F78             1
F79             3
F81           271
F82           181
F83           165
F84           463
F85           462
F86           394
F92           361
F93           349
F94           348
F95           328
F98             0
F99             0
F109            0
F110            0
F116            0
F121            0
F122            0
F123            0
F124            0
F125            0
F126            0
F127            0
F128            0
F129            0
F130            0
F131            0
F132            0
F133            0
F136            0
F139            0
COVID_TCPM      0
dtype: int64

In [5]:
test.isna().sum()

ID               0
F2               0
F3               0
F4               0
F8               3
F11              8
F25              0
F39             91
F52           1789
F53           1690
F54           1690
F55           1629
F69            113
F72           2286
F73           2179
F74           2102
F75           1925
F76             15
F77             15
F78             10
F79             11
F81           1314
F82            967
F83            908
F84           2293
F85           2286
F86           1970
F92           1780
F93           1685
F94           1685
F95           1625
F98              0
F99              0
F109             1
F110             1
F116             0
F121             0
F122             0
F123             0
F124             0
F125             0
F126             0
F127             0
F128             0
F129             0
F130             0
F131             0
F132             0
F133             0
F136             0
F139             0
COVID_TCPM       0
dtype: int64

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 52 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          500 non-null    int64  
 1   F2          500 non-null    int64  
 2   F3          500 non-null    int64  
 3   F4          500 non-null    int64  
 4   F8          500 non-null    object 
 5   F11         500 non-null    object 
 6   F25         500 non-null    float64
 7   F39         486 non-null    float64
 8   F52         136 non-null    float64
 9   F53         150 non-null    float64
 10  F54         151 non-null    float64
 11  F55         172 non-null    float64
 12  F69         477 non-null    float64
 13  F72         47 non-null     float64
 14  F73         63 non-null     float64
 15  F74         82 non-null     float64
 16  F75         111 non-null    float64
 17  F76         497 non-null    float64
 18  F77         497 non-null    float64
 19  F78         499 non-null    f

In [7]:
train[['F8', 'F11']]
train['F11'].value_counts()

,F8,F11
0,Tropical and Subtropical Moist Broadleaf Forests,"Mild temperate with dry winter, and Warm summer"
1,Temperate Broadleaf and Mixed Forests,"Mild temperate, fully humid, and Hot summer"
2,Temperate Broadleaf and Mixed Forests,"Mild temperate, fully humid, and Hot summer"
3,Tropical and Subtropical Moist Broadleaf Forests,Tropical monsoon
4,Temperate Broadleaf and Mixed Forests,"Snow, fully humid, and Warm summer"
...,...,...
495,Deserts and Xeric Shrublands,"Desert (arid), and Hot arid"
496,Temperate Broadleaf and Mixed Forests,"Snow, fully humid, and Hot summer"
497,Tropical and Subtropical Moist Broadleaf Forests,"Mild temperate, fully humid, and Hot summer"
498,Tropical and Subtropical Moist Broadleaf Forests,Tropical rain forest


Tropical savannah with dry winter                                           84
Mild temperate, fully humid, and  Warm summer                               70
Mild temperate with dry winter, and  Hot summer                             55
Mild temperate, fully humid, and  Hot summer                                50
Mild temperate with dry summer, and  Hot summer                             42
Desert (arid), and  Hot arid                                                36
Steppe (semi-arid), and  Hot arid                                           33
Tropical monsoon                                                            24
Steppe (semi-arid), and  Cold arid                                          22
Snow, fully humid, and  Warm summer                                         21
Tropical rain forest                                                        19
Snow, fully humid, and  Hot summer                                          13
Mild temperate with dry winter, and  Warm summer    

F8 - describes the land conditions of the specific region

F11 - describes the weather conditions of the specific region

In [8]:
na_percents = train.isna().sum()/len(train)*100
na_percents

ID             0.0
F2             0.0
F3             0.0
F4             0.0
F8             0.0
F11            0.0
F25            0.0
F39            2.8
F52           72.8
F53           70.0
F54           69.8
F55           65.6
F69            4.6
F72           90.6
F73           87.4
F74           83.6
F75           77.8
F76            0.6
F77            0.6
F78            0.2
F79            0.6
F81           54.2
F82           36.2
F83           33.0
F84           92.6
F85           92.4
F86           78.8
F92           72.2
F93           69.8
F94           69.6
F95           65.6
F98            0.0
F99            0.0
F109           0.0
F110           0.0
F116           0.0
F121           0.0
F122           0.0
F123           0.0
F124           0.0
F125           0.0
F126           0.0
F127           0.0
F128           0.0
F129           0.0
F130           0.0
F131           0.0
F132           0.0
F133           0.0
F136           0.0
F139           0.0
COVID_TCPM     0.0
dtype: float

In [9]:
cols_to_drop = na_percents[na_percents > 50.0].index.tolist()
train_copy_drops = train.drop(cols_to_drop, axis=1)
train_copy_drops.isna().sum() / len(train_copy_drops) * 100
train_copy_drops.isna().sum()

ID             0.0
F2             0.0
F3             0.0
F4             0.0
F8             0.0
F11            0.0
F25            0.0
F39            2.8
F69            4.6
F76            0.6
F77            0.6
F78            0.2
F79            0.6
F82           36.2
F83           33.0
F98            0.0
F99            0.0
F109           0.0
F110           0.0
F116           0.0
F121           0.0
F122           0.0
F123           0.0
F124           0.0
F125           0.0
F126           0.0
F127           0.0
F128           0.0
F129           0.0
F130           0.0
F131           0.0
F132           0.0
F133           0.0
F136           0.0
F139           0.0
COVID_TCPM     0.0
dtype: float64

ID              0
F2              0
F3              0
F4              0
F8              0
F11             0
F25             0
F39            14
F69            23
F76             3
F77             3
F78             1
F79             3
F82           181
F83           165
F98             0
F99             0
F109            0
F110            0
F116            0
F121            0
F122            0
F123            0
F124            0
F125            0
F126            0
F127            0
F128            0
F129            0
F130            0
F131            0
F132            0
F133            0
F136            0
F139            0
COVID_TCPM      0
dtype: int64

In [10]:
imputed_df = train_copy_drops[['F39', 'F69', 'F76', 'F77', 'F78', 'F79', 'F82', 'F83']].fillna(train_copy_drops[['F39', 'F69', 'F76', 'F77', 'F78', 'F79', 'F82', 'F83']].mean())
train_copy_drops[['F39', 'F69', 'F76', 'F77', 'F78', 'F79', 'F82', 'F83']] = imputed_df
train_copy_drops.isna().sum()
train_copy_drops["F11"].value_counts()

ID            0
F2            0
F3            0
F4            0
F8            0
F11           0
F25           0
F39           0
F69           0
F76           0
F77           0
F78           0
F79           0
F82           0
F83           0
F98           0
F99           0
F109          0
F110          0
F116          0
F121          0
F122          0
F123          0
F124          0
F125          0
F126          0
F127          0
F128          0
F129          0
F130          0
F131          0
F132          0
F133          0
F136          0
F139          0
COVID_TCPM    0
dtype: int64

Tropical savannah with dry winter                                           84
Mild temperate, fully humid, and  Warm summer                               70
Mild temperate with dry winter, and  Hot summer                             55
Mild temperate, fully humid, and  Hot summer                                50
Mild temperate with dry summer, and  Hot summer                             42
Desert (arid), and  Hot arid                                                36
Steppe (semi-arid), and  Hot arid                                           33
Tropical monsoon                                                            24
Steppe (semi-arid), and  Cold arid                                          22
Snow, fully humid, and  Warm summer                                         21
Tropical rain forest                                                        19
Snow, fully humid, and  Hot summer                                          13
Mild temperate with dry winter, and  Warm summer    

In [11]:
train_copy_drops['F8'] = train_copy_drops['F8'].astype('category')
train_copy_drops['F11'] = train_copy_drops['F11'].astype('category')
train_copy_drops['F136'] = train_copy_drops['F136'].astype('category')
df_train_onehot = pd.get_dummies(train_copy_drops, columns=['F8', 'F11', 'F136'], prefix=['F8_Area_Condition', 'F11_Climate', 'F136'])
df_train_onehot.head(5)

,ID,F2,F3,F4,F25,F39,F69,F76,F77,F78,...,"F11_Climate_Steppe (semi-arid), and Cold arid","F11_Climate_Steppe (semi-arid), and Cold arid;Snow with dry summer, and Hot summer","F11_Climate_Steppe (semi-arid), and Hot arid",F11_Climate_Tropical monsoon,F11_Climate_Tropical rain forest,F11_Climate_Tropical savannah with dry summer,F11_Climate_Tropical savannah with dry winter,F136_available,F136_missing,F136_preliminary
0,10681,1,1,1,90282.49890,0.367070,83.449504,4988.544333,5949.288232,6872.472783,...,0,0,0,0,0,0,0,1,0,0
1,3300,1,1,1,288522.81830,0.420663,65.779902,42724.435510,19536.915530,85912.946920,...,0,0,0,0,0,0,0,1,0,0
2,12038,1,1,1,94697.53400,0.343825,48.131525,9826.923968,11719.510030,13461.595530,...,0,0,0,0,0,0,0,1,0,0
3,10236,0,1,1,175315.10010,0.558702,9.809284,3483.049562,4605.716581,5039.788915,...,0,0,0,1,0,0,0,0,0,1
4,4111,1,1,1,89094.63635,0.508010,3.993893,1599.962445,863.509687,240.446778,...,0,0,0,0,0,0,0,1,0,0


In [12]:
X_train = df_train_onehot.drop('COVID_TCPM', axis=1)
Y_train = df_train_onehot['COVID_TCPM']
X_train.isna().sum()

ID                                               0
F2                                               0
F3                                               0
F4                                               0
F25                                              0
                                                ..
F11_Climate_Tropical savannah with dry summer    0
F11_Climate_Tropical savannah with dry winter    0
F136_available                                   0
F136_missing                                     0
F136_preliminary                                 0
Length: 72, dtype: int64

In [13]:
le = LabelEncoder()
le.fit(Y_train)
le.classes_

LabelEncoder()

array(['high', 'low'], dtype=object)

In [14]:
clf_dt = dt(criterion='gini', splitter='best', max_depth=6, min_samples_split=10, min_samples_leaf=5, max_features='auto')
clf_dt = clf_dt.fit(X_train, Y_train)

In [15]:
clf_dt.score(X_train, Y_train)

0.87

In [19]:
no_nan_test = pd.read_csv('clean_test.csv')
no_nan_test['F8'] = no_nan_test['F8'].astype('category')
no_nan_test['F11'] = no_nan_test['F11'].astype('category')
no_nan_test['F136'] = no_nan_test['F136'].astype('category')
df_test_onehot = pd.get_dummies(no_nan_test, columns=['F8', 'F11'], prefix=['F8_Area_Condition', 'F11_Climate'])
df_test_onehot.head(5)
df_test_onehot.shape

,ID,F2,F3,F4,F25,F39,F69,F76,F77,F78,...,"F11_Climate_Snow, fully humid, and Cool summer","F11_Climate_Snow, fully humid, and Hot summer","F11_Climate_Snow, fully humid, and Warm summer","F11_Climate_Steppe (semi-arid), and Cold arid","F11_Climate_Steppe (semi-arid), and Hot arid",F11_Climate_Tropical monsoon,F11_Climate_Tropical rain forest,F11_Climate_Tropical savannah with dry summer,F11_Climate_Tropical savannah with dry winter,F11_Climate_Tundra
0,12272.0,1.0,1.0,1.0,301011.40070,0.270918,57.889697,45187.287610,53889.947660,61039.963870,...,0,0,0,0,0,0,0,0,0,0
1,6526.0,1.0,1.0,1.0,86680.35925,0.303313,21.221158,86.386399,109.933379,129.268833,...,0,0,0,0,0,0,0,0,0,0
2,11320.0,2.0,1.0,1.0,297267.15000,0.359829,86.350782,63083.100680,75232.254990,97333.212900,...,0,0,0,0,0,0,0,0,0,0
3,4566.0,0.0,0.0,1.0,92949.66541,0.274426,7.550097,2205.528121,2213.771912,2076.353124,...,0,0,0,0,0,0,0,0,0,0
4,4866.0,1.0,1.0,1.0,154643.44400,0.475473,0.018488,697.059179,1023.343533,1299.903914,...,0,0,0,0,0,0,0,0,0,0


(2486, 77)

In [17]:
no_nan_test['F11']

0         available
1         available
2         available
3         available
4         available
           ...     
2481      available
2482    preliminary
2483      available
2484      available
2485      available
Name: F11, Length: 2486, dtype: category
Categories (3, object): [available, missing, preliminary]

In [18]:
train_cols = set(X_train.columns.tolist())
test_cols = set(df_test_onehot.columns.tolist())
test_cols - train_cols

{'F11_Climate_available',
 'F11_Climate_missing',
 'F11_Climate_preliminary',
 'F136',
 'F8_Area_Condition_Temperate Coniferous Forests;Deserts and Xeric Shrublands',
 'F8_Area_Condition_Tropical and Subtropical Dry Broadleaf Forests;Tropical and Subtropical Coniferous Forests',
 'F8_Area_Condition_Tropical and Subtropical Moist Broadleaf Forests;Mangroves',
 'F8_Area_Condition_Tropical and Subtropical Moist Broadleaf Forests;Tropical and Subtropical Dry Broadleaf Forests',
 'F8_Area_Condition_Tundra'}